In [248]:
import json

import pickle
import numpy as np

In [249]:
def load_data_json(json_path):
    f = open(json_path, 'r', encoding='utf-8')
    json_data = json.load(f)
    f.close()
    return json_data

In [250]:
json_path = f'./data/train_complete.json'
json_data = load_data_json(json_path)

results_path = f'./output_results/train_result_colbert30'
with open(results_path, 'rb') as handle:
    results = pickle.load(handle)
            

In [251]:
print(len(json_data))

560


In [252]:
not_in_count = 0
not_in_dids = []
for data in json_data:
    pos_dids = data['pos_dids']
    bm25_pos_dids = data['bm25_pos_dids']
    
    did = data['did']
    for pos_did in pos_dids:
        if pos_did not in bm25_pos_dids:
            not_in_count += 1
            not_in_dids.append(did)
            break
            

In [253]:
not_in_count

40

In [254]:
def softmax(x):
    """Compute the softmax in a numerically stable way."""
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [255]:
from sklearn.metrics import f1_score, recall_score, precision_score

from sklearn.preprocessing import MultiLabelBinarizer
multi = MultiLabelBinarizer()

all_dids = []
did2data = {}
for d in json_data:
    did = d['did']
    did2data[did] = d
    all_dids.append(did)


In [265]:
topk = 20
avg_pos_softmax_score = 0.0
c = 0
avg_pos_hit_rate = 0

y = []
y_bm25 = []
y_pred = []

train_binary_pairs = []
train_pos_count = 0
train_neg_count = 0

for k, values in results.items():
    did = str(k)
    
    data = did2data[did]
    pos_dids = data['pos_dids']
    
    if len(pos_dids) > 0:
        for pos_did in pos_dids:
            train_binary_pairs.append((did, pos_did, 1))
            train_pos_count += 1
#     print(f'did = {k}')
    
    bm25_pos_dids = data['bm25_pos_dids'][:topk]
    
    all_ref_ids = np.array([v[0] for v in values])
    scores = [v[2] for v in values]
    scores = np.array(scores)
    
    
        
    mean = np.mean(scores)
    std = np.std(scores)
    max_score = np.max(scores)
    
#     print(f' mean {mean}, std {std}, max {max_score}')
    
#     for pos_did in pos_dids :
#         idx = np.where(all_ref_ids == pos_did)
#         print(f'did : {all_ref_ids[idx]}, score : {scores[idx]}')
        

    
    th = mean + 1.5 * std
    pos_idx = scores >= th

    
#     scores = softmax(scores)
#     topk_idx = scores.argsort()[::-1][:topk]
    
#     binary_scores = [v[1] for v in values]
#     binary_scores = np.array(binary_scores)

#     print('bm25 pos dids')
#     print(bm25_pos_dids)
    
#     print('score based predict topk did and score')
    preds = list(all_ref_ids[pos_idx])
#     print(preds)
    
    if len(pos_dids) > 0:
        for pred in preds:
            if pred in pos_dids:
                continue
            train_binary_pairs.append((did, pred, 0))
            train_neg_count += 1

    
        print(len(pos_dids), len(pred), len(pos_dids)/(len(pred) + len(pos_dids)))

    y.append(pos_dids)
    y_bm25.append(bm25_pos_dids)
    y_pred.append(preds)




2 4 0.3333333333333333
1 4 0.2
2 2 0.5
1 4 0.2
1 4 0.2
1 3 0.25
2 2 0.5
1 4 0.2
3 4 0.42857142857142855
1 4 0.2
2 4 0.3333333333333333
5 4 0.5555555555555556
1 4 0.2
9 4 0.6923076923076923
4 3 0.5714285714285714
9 4 0.6923076923076923
4 3 0.5714285714285714
2 4 0.3333333333333333
4 3 0.5714285714285714
2 4 0.3333333333333333
3 4 0.42857142857142855
1 4 0.2
1 3 0.25
1 3 0.25
2 4 0.3333333333333333
1 3 0.25
1 3 0.25
1 4 0.2
1 4 0.2
9 4 0.6923076923076923
1 3 0.25
1 4 0.2
1 4 0.2
1 4 0.2
12 4 0.75
11 4 0.7333333333333333
3 4 0.42857142857142855
3 3 0.5
1 4 0.2
1 4 0.2
1 4 0.2
1 4 0.2
11 4 0.7333333333333333
3 3 0.5
2 3 0.4
1 3 0.25
8 4 0.6666666666666666
2 3 0.4
3 4 0.42857142857142855
1 4 0.2
2 3 0.4
1 4 0.2
3 4 0.42857142857142855
1 3 0.25
1 4 0.2
2 4 0.3333333333333333
8 4 0.6666666666666666
3 4 0.42857142857142855
2 4 0.3333333333333333
10 4 0.7142857142857143
2 4 0.3333333333333333
1 4 0.2
1 4 0.2
1 3 0.25
1 4 0.2
8 4 0.6666666666666666
1 4 0.2
1 3 0.25
1 3 0.25
1 3 0.25
3 3 0.5
1 4 

In [266]:
# print(len(y))
# print(len(y_bm25))
# print(len(y_pred))

multi = MultiLabelBinarizer()
y_multi = multi.fit_transform(y)
y_bm25_multi = multi.transform(y_bm25)
y_pred_multi = multi.transform(y_pred)

# print(y_multi.shape)
# print(y_bm25_multi.shape)
# print(y_pred_multi.shape)

average = 'macro'
bm25_f1  = f1_score(y_multi, y_bm25_multi ,average=average)
bm25_recall  = recall_score(y_multi, y_bm25_multi ,average=average)
bm25_precision  = precision_score(y_multi, y_bm25_multi ,average=average)

print('bm25')
print(f'f1:{bm25_f1}, recall:{bm25_recall}, precision:{bm25_precision} ')

mode = 'macro'
pred_f1  = f1_score(y_multi, y_pred_multi ,average=average)
pred_recall  = recall_score(y_multi, y_pred_multi ,average=average)
pred_precision  = precision_score(y_multi, y_pred_multi ,average=average)

print('colbert')
print(f'f1:{pred_f1}, recall:{pred_recall}, precision:{pred_precision} ')

bm25
f1:0.19893197205110855, recall:0.6964530596811046, precision:0.1404706100041387 
colbert
f1:0.3637315212088835, recall:0.8369960124813757, precision:0.27990555870238276 


/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:860: UserWarning: unknown class(es) ['1', '1011', '1016', '105', '1056', '1068', '1082', '1086', '1087', '1093', '110', '1118', '1128', '113', '1130', '1132', '1142', '1146', '1147', '1148', '1151', '1153', '1155', '1157', '1159', '1161', '1165', '117', '1185', '119', '1191', '1206', '121', '1239', '1261', '1301', '1329', '1332', '1334', '1341', '1352', '1354', '1364', '1365', '1366', '1375', '138', '1390', '14', '140', '1401', '142', '147', '150', '152', '155', '157', '178', '184', '185', '189', '199', '201', '209', '21', '215', '219', '225', '242', '248', '26', '262', '263', '267', '268', '273', '283', '295', '297', '299', '3', '301', '303', '310', '311', '313', '319', '329', '333', '337', '342', '343', '351', '365', '370', '371', '381', '387', '39', '392', '403', '411', '421', '441', '449', '453', '458', '463', '465', '471', '472', '474', '484', '501', '504', '508', '51', '510', '516', '522', '5

In [267]:
print(len(train_binary_pairs))
print(train_pos_count, train_neg_count)
print(train_pos_count / len(train_binary_pairs))
train_binary_pairs[:10]

5150
1383 3767
0.26854368932038836


[('3', '415', 1),
 ('3', '649', 1),
 ('3', '107', 0),
 ('3', '189', 0),
 ('3', '636', 0),
 ('3', '661', 0),
 ('3', '665', 0),
 ('3', '684', 0),
 ('3', '689', 0),
 ('3', '709', 0)]

In [268]:
import pickle
with open('./data/train_binary_pairs', 'wb') as handle:
    pickle.dump(train_binary_pairs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
pos_pairs = []
th = 0.15
topk = 10

for k, values in results.items():
    did = str(k)
    print(f'did = {k}')
    data = did2data[did]
    pos_dids = data['pos_dids']
    bm25_pos_dids = data['bm25_pos_dids'][:10]
    
    all_ref_ids = np.array([v[0] for v in values])
    scores = [v[2] for v in values]
    scores = np.array(scores)
    scores = softmax(scores)
    
#     pos_idx = scores>th
#     pos_ref_ids = all_ref_ids[pos_idx]

    pos_idx = scores.argsort()[::-1][:topk]
    pos_ref_ids = all_ref_ids[topk_idx]
    
    
    
    for pos_ref_id in pos_ref_ids:
        pos_pairs.append((did, pos_ref_id))


did = 7
did = 13
did = 19
did = 20
did = 24
did = 28
did = 30
did = 31
did = 38
did = 42
did = 45
did = 50
did = 52
did = 56
did = 60
did = 69
did = 71
did = 74
did = 79
did = 80
did = 84
did = 87
did = 90
did = 92
did = 96
did = 102
did = 109
did = 111
did = 112
did = 114
did = 116
did = 122
did = 123
did = 124
did = 125
did = 128
did = 130
did = 132
did = 134
did = 137
did = 143
did = 145
did = 148
did = 153
did = 159
did = 160
did = 161
did = 162
did = 164
did = 165
did = 168
did = 172
did = 175
did = 177
did = 187
did = 190
did = 192
did = 193
did = 195
did = 198
did = 200
did = 202
did = 206
did = 213
did = 214
did = 216
did = 229
did = 233
did = 234
did = 244
did = 245
did = 247
did = 249
did = 250
did = 254
did = 257
did = 264
did = 265
did = 269
did = 270
did = 272
did = 277
did = 281
did = 287
did = 288
did = 290
did = 293
did = 294
did = 304
did = 309
did = 312
did = 315
did = 324
did = 331
did = 334
did = 335
did = 339
did = 345
did = 350
did = 352
did = 358
did = 359
did = 

In [40]:
len(pos_pairs)

4210

In [41]:
import pandas as pd
submit_path = './submit/top10.csv'
df = pd.DataFrame(pos_pairs, columns=['Test','Reference'])
df.to_csv(submit_path, index=None)

,Test,Reference
0,13,410
1,13,416
2,20,24
3,24,499
4,28,30
...,...,...
524,1385,987
525,1389,130
526,1393,423
527,1398,427
